In [1]:
import pandas as pd
from pydantic import BaseModel, field_validator, ValidationError
import os
from datetime import  datetime
import phonenumbers
import os,tabulate
import numpy as np
import random



In [2]:
def loading_dataset(filename,filename1,filename2,filename3,filename4):
    path = os.path.abspath('C:/Users/Balli/Pictures/')
    if path:
        fullpath = os.path.join(path,filename)
        fullpath1 = os.path.join(path,filename1)
        fullpath2 = os.path.join(path,filename2)
        fullpath3 = os.path.join(path,filename3)
        fullpath4 = os.path.join(path,filename4)

        dataset_weblog = pd.read_csv(fullpath)
        dataset_ipinfo = pd.read_csv(fullpath1)
        dataset_product= pd.read_csv(fullpath2)
        dataset_transaction= pd.read_csv(fullpath3)
        dataset_customer = pd.read_csv(fullpath4)

        df = dataset_weblog.copy()
        df1 = dataset_ipinfo.copy()
        df2 = dataset_product.copy()
        df3 = dataset_transaction.copy()
        customer_dataset = dataset_customer.copy()

#perform a join querry of our two dataset that are related 
        weblog_ipinfo_dataset = pd.merge(df,df1, on='IP',how='left')
        transactn_product_dataset = pd.merge(df3,df2, on='ProductID',how='left')
        return weblog_ipinfo_dataset,transactn_product_dataset,df,customer_dataset
        



In [3]:

def model_creation():
    class customerweb_infomodel(BaseModel):
        timespent_perday: int
        no_pages_visited: int
        location: str
        hour_session_starts: int
        was_there_purchase: int
        
# creating validation for column number of pages visited
        @field_validator('no_pages_visited')
        def check_pages_visited(cls, value):
            if value <= 0:
                raise ValidationError('Pages visited field can not be less than 0')
            else:
                return value
            
# creating validation for column was there a purchase to convert integer to a string answer of Yes or No 
        @field_validator('was_there_purchase')
        def purchase(cls, value):
            if value == 1:
                return 'Yes'
            else:
                return 'No'

    class transaction_model(BaseModel):
        productname:str
        price: float
        date: datetime
        hour_of_transaction: int

        @field_validator('price')
        def check_price(cls, value):
            if value:
                return f'${value}'
            elif value <= 0:
                raise ValidationError('Price must be greater than 0')
            else:
                return None
            

    class webpages_visitedmodel(BaseModel):
        last_web_pagevisited: str
        no_of_time_pagevisited: int

    return {
        'customerweb_infomodel': customerweb_infomodel,
        'transaction_model': transaction_model,
        'webpages_visitedmodel': webpages_visitedmodel}



In [4]:
def customerweb_record(df):           
        model = model_creation()
        customerweb_infomodel = model['customerweb_infomodel']        
        if not df.empty:        
                df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

        # filtering the date out of the datetime
                df['date'] = df['timestamp'].dt.date

        # filtering the hour of user session out of datetime
                df['hour'] = df['timestamp'].dt.hour

                
                customerweb_recordtable=df.groupby(['UserId','date','hour','Location','product_to_cart']).agg(
                                                timespentdaily=('Duration','sum'),
                                                pages_visited=('Address','count')).reset_index()
                
                customerweb_recordtable.drop_duplicates(subset=['UserId'])
                

        # defining a variable list to store data for the model customer_web_info       
                data_list = []
        # Iterating over the customer web record table to load our model with data
                
                for _, row in customerweb_recordtable.iterrows():
                        try:    
                                customerweb_table = customerweb_infomodel(timespent_perday=row['timespentdaily'] ,
                                                        no_pages_visited=row['pages_visited'],
                                                        location=row['Location'],
                                                        hour_session_starts=row['hour'],
                                                        was_there_purchase=row['product_to_cart'])
                                
                                data_list.append(customerweb_table.model_dump())
                        except Exception as error:
                                print(f'error:{error}')

        # converting data list to pandas dataframe
                if data_list:
                        customerweb_record_df=pd.DataFrame(data_list)
                        print(tabulate.tabulate(customerweb_record_df.set_index('timespent_perday'),headers=customerweb_record_df.columns.tolist(),tablefmt = 'heavy_grid'))
                
        #saving our file/dataset as csv
                customerweb_record_df.to_csv('Customer web record.csv', index=False)
                print('Customer web record saved successfully')



In [5]:
def transactn_record(df):
    model = model_creation()
    transaction_model = model['transaction_model']
    if not df.empty:
    
# converting to datetime to be able to get the date and hour of  daily transaction
        df['transactn_time'] = pd.to_datetime(df['time_transactn'],unit='s')

# filtering the date out of the datetime
        df['date'] = df['transactn_time'].dt.date

#filtering the hour of transaction out of datetime
        df['hour'] = df['transactn_time'].dt.hour

#creating a dictionary of our new formated dataset
        DictData={'date':df['date'],
                'houroftransactn':df['hour'],
                'productname':df['Productdescriptn'],
                'price':df['price']}

#converting to pandas dataframe
        final_dataset=pd.DataFrame(DictData)
    
 #stacking final engineered dataset into the transaction table or model earlier defined above

        transaction_datalist = []
        # iterating over finaldata1 to load our model
        for _, row in final_dataset.iterrows():
            transaction_table = transaction_model(productname=row['productname'],
                                                date= row['date'], 
                                                hour_of_transaction=row['houroftransactn'], 
                                                price=row['price'])
            
            transaction_datalist.append(transaction_table.model_dump())
# converting transaction datalist to pandas dataframe
        if transaction_datalist:
                transactn_df=pd.DataFrame(transaction_datalist)
                print(tabulate.tabulate(transactn_df.set_index('productname'),headers=transactn_df.columns.tolist(),tablefmt = 'heavy_grid'))
                # saving our data table as csv
                transactn_df.to_csv('transaction record.csv', index=False)
                print('transaction record saved successfully')
            

In [6]:
def lastwebpage_visited_no(df):
    model = model_creation()
    webpages_visitedmodel = model['webpages_visitedmodel']
    if not df.empty:
        df.fillna({'Link': 'Link unavailable'},inplace = True)

# Group by the Address column and count occurrences
        webpages_visitedtimes_df = df.groupby('Address').size().reset_index(name='count')

#set a list variable to store our table data
        datatable = []

#iterate over the last_page_count table and load into our model
        for _, row in webpages_visitedtimes_df.iterrows():
            model_data=webpages_visitedmodel(last_web_pagevisited=row['Address'],
                                                no_of_time_pagevisited=row['count'])
            datatable.append(model_data.model_dump())
#convert our data table 3 into pandas dataframe
        if datatable:
            lastweb_visited_no=pd.DataFrame(datatable)
            print(tabulate.tabulate(lastweb_visited_no.set_index('last_web_pagevisited'),headers=lastweb_visited_no.columns.tolist(),tablefmt = 'heavy_grid'))
# saving our file/data  as csv
            lastweb_visited_no.to_csv('Customer last webpage vistedcount.csv', index=False)
            print(' Document saved successfully')


In [ ]:
def promotion_record(df):

    if not df.empty:

        # The Gender column is empty, and since there is no module in python currently to decipher
        # male or female from the names. it became sacrosanct to use Excel and manually fill this column since
        # we can decipher male names from female names by bare looking at it.

        # now let us deal wit outliers
        # from the dataset it can be noticed that the age column have an outlier which is 500, because no human in the generation
        # can live to 500 years

        # because this dataset is no many and we can easily spot outliers so we may not need to use IQR method but directly
        # replace the outlier with the mean or median or mode as the case may be, but for this dataset, i may have to use mean
        
        # Now i am going t replace the outlier with the mean_age
        df['age'] = df['age'].apply( lambda x: df['age'].mean() if x >= 500 else x)
        
# replacing the null value in the age column with the median age
        df.fillna({'age':df['age'].median()},inplace=True)
        df['age'] = df['age'].astype(int)

# Now cleaning the Education column and replacing the null with the most frequent education category

        df['education'].value_counts(dropna=False)
        df.fillna({'education':df['education'].mode()[0]},inplace=True)
# stripping off excess whitespaces in the Education column
        df['education']= df['education'].str.strip()

# Now cleaning the income column to replace the null with the mean income and adding the $ sign
        
        df.fillna({'income':round(df['income'].mean(),2)},inplace=True)
        def add_currencySign(income):
            if income:
                return '${:,}'.format(income)
            else:
                return None
        df['income']= df['income'].apply(add_currencySign)

# Now cleaning the Consent column to replace any string to integer and convert datatype to int
        df['consent']= df['consent'].apply(lambda x: 1 if x == 'i' else x).astype(int)

# Cleaning the phone number column and also creating a link for one-time click dial Filling any phone number that is less 10 digit

        df['phone']= df['phone'].apply(lambda x: int(random.randint(10**9,10**10)) if pd.isnull(x) else int(x)).astype(str).str.zfill(10)

# Trimming any phone number that is greater than 10 digits
        df['phone'] = df['phone'].str.slice(0,9)

# Now converting the phone number column to proper phone number standard using the python phonenumber module
        def format_phone_number(phone):
            try:
                # Parse the phone number
                parsed_number = phonenumbers.parse(phone, "CA")
                # Format the phone number in E.164 format
                formatted_number = phonenumbers.format_number(parsed_number, phonenumbers.PhoneNumberFormat.E164)
                return formatted_number
            except phonenumbers.NumberParseException:
                return phone 
            
# Now applying the function format_phone_number to the phone column of our dataset
        df['phone']= df['phone'].apply(format_phone_number)

# Final step is to create a link on the phone column so that this can aid auto-dail by creating a function

        def clickable_phone(phone):
            if phone:
                return f'<a href="{phone}">{phone}</a>'
            else:
                return None

        df['phone']= df['phone'].apply(clickable_phone)

        print(tabulate.tabulate(df.set_index('name'),headers=df.columns.tolist(),tablefmt = 'heavy_grid'))

# To enable the customer rep to dial customer phone numbers automatically, we need to save te dataset in an html format
# this will enable the dataset to e opened through a browser and then giving the link to auto dial a number

        df.to_html('customerdataset.html',escape=False,index=False)
        print('dataset saved in html file successfully')



In [8]:
weblog_ipinfo,transactn_product,weblog,customer = loading_dataset('store_web_log.csv','ip_info.csv','store_product_table.csv','store_transaction_records.csv','customerdata.csv')
customerweb_activity = customerweb_record(weblog_ipinfo)
transactionrecord = transactn_record(transactn_product)
customer_lastwebpage_visited = lastwebpage_visited_no(weblog)
promotion_calls = promotion_record(customer)

c:\Users\Balli\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\Balli\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃   timespent_perday ┃   no_pages_visited ┃ location                  ┃   hour_session_starts ┃ was_there_purchase   ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━┫
┃                195 ┃                  3 ┃ Ontario, Canada           ┃                    19 ┃ No                   ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━┫
┃                 41 ┃                  1 ┃ Ontario, Canada           ┃                    19 ┃ Yes                  ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━━┫
┃                272 ┃                  3 ┃ Texas, United States      ┃                    19 ┃ No                   ┃
┣━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━╋━━━━━